# Plot XRD maps from flyscans

This notebook will plot an XRD map from a flyscan. It requires data from the `.h5` masterfile and the `_pilatus_integrated.h5` file.

A ROI for the XRD map (in radial bins) should be defined in one of the cell below.

You can use a scan database to get the scan numbers based on a sample name rather than numbers.
To do this, you need to make a database using the mapping notebook "Samples_list"
This database can also be done to create and save images in bulk

### Load data and plot average XRD for the map

In [ ]:
%matplotlib widget
import os
import sys
import h5py
import numpy as np
import matplotlib.pyplot as plt
#To import DanMAX from the folder above:
sys.path.append('../')
import DanMAX as DM
style = DM.darkMode(style_dic={'figure.figsize':'large'})

In [ ]:
# Define scan location:
#Note that scans must be a list!

samples_database = False
if samples_database: # To use samples_database, and to do bulk processing, use the "Samples_list" notebook
    groups = DM.getProposalScans()
    group = 'group'
    sample = 'sample'
    scans = groups[group][sample]
else:
    scans = [XXXX]
proposal,visit=DM.getCurrentProposal()

#Select ranges to load (in the unit the data were integrated with)
#useful for reducing the size of large datasets
xrd_range = None
azi_range = None

#Load data
maps = DM.mapping.stitchScans(scans,XRF=False,proposal=proposal,visit=visit,xrd_range=xrd_range,azi_range=azi_range)
#Apply I0 correction
xrd_map = (maps['xrd_map'].transpose(2,0,1)/ maps['I0_map']).transpose(1,2,0)
xrd_avg = np.mean(maps['xrd_map'],axis=(0,1))


### Plot avarage XRD map

In [ ]:
#Set ROIs:

# define ROI dictionary with a label str and a tuple of lower and upper limits, given in the scattering units (2theta or Q)
#     'label': (lower,upper)
xrd_rois = {'peak_1' : (  XXX, XXX),
        'peak_2' : (XXX,XXX)}

# plot average XRD pattern
plt.figure()
plt.title('Average XRD pattern')
plt.plot(maps['x_xrd'],xrd_avg)
print(maps['xrd_map'].dtype)
if maps['Q']:
    plt.xlabel('Q (A-1)')
else:
    plt.xlabel('2theta (deg)')
plt.ylabel('Intensity')
plt.yscale('log')

for key in xrd_rois:
    xrd_roi = (maps['x_xrd'] > xrd_rois[key][0]) & (maps['x_xrd'] < xrd_rois[key][1])
    plt.axvline(xrd_rois[key][0],c='k',ls='--',lw=1)
    plt.axvline(xrd_rois[key][1],c='k',ls='--',lw=1)
    plt.plot(maps['x_xrd'][xrd_roi],xrd_avg[xrd_roi], '.',ms=2,label=key)
plt.legend()

### Plot XRD map
It is probably nescessesary to change the vmax value

In [ ]:
# Set the number of columns for the figure
cols = 2

rows = int(len(xrd_rois)/cols) + (len(xrd_rois)%cols!=0)
# initialize subplots with shared x- and y-axes
fig,axs = plt.subplots(rows,cols,sharex=True,sharey=True)
fig.tight_layout()
axs = axs.flatten() # flatten the axes list to make it easier to index

I_xrd = []
for i,key in enumerate(xrd_rois):
    lower, upper = xrd_rois[key]
    lower, upper = np.digitize(lower, maps['x_xrd']), np.digitize(upper, maps['x_xrd'])
    
    #calculate background
    bkg = np.mean([np.mean(xrd_map[:,:,lower-5:lower],axis=2), np.mean(xrd_map[:,:,upper+1:upper+6],axis=2)],axis=0)
    # calculate the average diffraction peak signal from the integrated XRD data
    xrd_map_bgr = np.trapz((xrd_map[:,:,lower:upper].transpose(2,0,1) - bkg).transpose(1,2,0),maps['x_xrd'][lower:upper])
    xrd_map_bgr -=np.nanmin(xrd_map_bgr)
                           
    I_xrd.append(xrd_map_bgr)
    

    # plot the map as a pseudo colormesh
    ax = axs[i]
    ax.set_title(key)
    pcm = ax.pcolormesh(maps['x_map'],
                        maps['y_map'],
                        I_xrd[i],
                        shading='nearest')
    fig.colorbar(pcm,ax=ax,aspect=40,pad=0.05)
    ax.set_xlabel('x mm')
    ax.set_ylabel('y mm')
    # set the aspect ratio to equal to give square pixels
    ax.set_aspect('equal')

# delete surplus plots
for i in range(1,cols*rows-len(xrd_rois)+1):
    fig.delaxes(axs[-i])

# Bulk XRD Maps.

Will create a map of every roi for every scan in your database.
If you don't have a database, you can create one using the "Samples_list.ipynb" notebook

In [ ]:
peaks = {
    'peak_1': {
        'xrd_range': [7.95,8.55], # tth/q values for the peak of interest Including background
        'bkgtth': np.r_[0:6,14:20], # indexes from background
        'datatth': np.s_[6:14], #Indexes for data in loaded data.
        'scale_bar': True, # Use scale bar in saved png images
        'scale_place': [[10,15],[10,49]], # Location of scale bar in the image, indexes
        'limits': [0, 1] # vmin and vmax to use when saving as png
        'save_h5': True # Save maps in a combined h5 file
        'save_png': True# # Save maps individually in png files
    },
}

In [ ]:
groups = DM.getProposalScans()
proposal,visit=DM.getCurrentProposal()
file_written = False
samples_database = False
if not samples_database:
    break
def _save_im(im,save_folder, save_file,grp = None):
    save_folder = f'{process_folder}/'
    if not os.path.isdir(save_folder):
        os.makedirs(save_folder)
        if not grp is None:
            os.chown(save_folder, -1, grp)
        os.chmod(save_folder,0o770)
    plt.imsave(save_file,im)
    if not grp is None:
        os.chown(save_file, -1, grp)
    os.chmod(save_file,0o770)

for peak in peaks:
    print(f'Making figures of peak: {peak}')
    xrd_range = peaks[peak]['xrd_range']
    bkgtth = peaks[peak]['bkgtth']
    datatth = peaks[peak]['datatth']
    scale_bar = peaks[peak]['scale_bar']
    scale_place = peaks[peak]['scale_place']
    limits = peaks[peak]['limits']
    save_h5 = peaks[peak]['save_h5']
    save_png = peaks[peak]['save_png']
    for group in groups:
        print(f'\tMaking figures for group: {group}')
        file_exists = False
        for sample in groups[group]:
            print(f'\t\tFigures for: {sample}')
            
            maps = DM.mapping.stitchScans(scans,XRF=False,proposal=proposal,visit=visit,xrd_range=xrd_range,azi_range=None)
            xrd_map = (maps['xrd_map'].transpose(2,0,1)/ maps['I0_map']).transpose(1,2,0)
            
            
            bkg = np.mean(xrd_map[:,:,bkgtth],axis=2)
            # calculate the average diffraction peak signal from the integrated XRD data
            xrd_map = np.trapz((xrd_map[:,:,data_tth].transpose(2,0,1) - bkg).transpose(1,2,0),maps['x_xrd'][data_tth])
            
            del maps
            xrd_map -=np.nanmin(xrd_map_bgr)
            
            
            scans = groups[group][animal]
            process_folder = DM.getAzintFname(DM.findScan(groups[group][animal][0])).split('/azint/')[0]

            
            grp = os.stat(process_folder).st_grp
            maps_folder = f'{process_folder}/xrd_maps'
            

            xrd_maps_file_png =f'{process_folder}/xrd_maps/{group}/{sample}/peak_{peak}.png'
            xrd_maps_file = f'{process_folder}/xrd_maps/xrd_maps.h5'
            if save_h5:
                if not os.path.isdir(maps_folder):
                    os.makedirs(maps_folder)
                    os.chown(maps_folder,-1,grp)
                    os.chmod(maps_folder,0o770)
                    
                if not file_written:
                    with h5py.File(xrd_maps_file,'w') as xmf:
                        xmf.create_dataset(f'{group}/{sample}/{peak}',data= xrd_map)
                    file_written = True

                with h5py.File(xrd_maps_file, 'a') as xmf:
                    xmf.create_dataset(f'{group}/{sample}/{peak}',data= xrd_map)
                
            if save_png:
                
                xrd_map_im = DM.mapping.mat_to_im(
                    xrd_map,
                    limits[0],
                    limits[1],
                    'viridis',
                    True,
                    scale_place
                )
                _save_im(xrd_map_im,xrd_maps_folder,xrd_maps_file_png,grp)